# *Salix* and *Populus* sequence capture: Assembly and Gene Trees

Brian J. Sanderson

Last updated: 7 May 2020

Required programs: [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic), [HybPiper](https://github.com/mossmatters/HybPiper), [mafft](https://mafft.cbrc.jp/alignment/software/), [pal2nal](http://www.bork.embl.de/pal2nal/), [trimal](http://trimal.cgenomics.org/), [RAxML](https://cme.h-its.org/exelixis/software.html)

---

This analysis notebook describes the assembly of gene sequences and estimation of gene trees from a trial set of three *Populus* and three *Salix* species using our targeteds seqeunce capture array. Included below are a number of HPCC submission scripts. I include these here because when I first started working on this project it took some time for me to adapt existing pipelines I found to work on an HPCC. And so, even though having in-line submission scripts is not the most "readable" I am hoping it will help save someone else time in the future.

## Trim and QC Reads

1. Run Trimmomatic to trim low quality and short reads

   **note** files.txt contains just the prefixes of each pair of read files in the raw/ directory

```bash
parallel --eta --gnu -j 20 "java -jar ~/bin/Trimmomatic-0.36/trimmomatic-0.36.jar \
                            PE -phred33 \
                            raw/{}_L001_R1_001.fastq \
                            raw/{}_L001_R2_001.fastq \
                            trimmed/{}_R1_paired.fastq \
                            trimmed/{}_R1_unpaired.fastq \
                            trimmed/{}_R2_paired.fastq \
                            trimmed/{}_R2_unpaired.fastq \
                            ILLUMINACLIP:TruSeq3-PE.fa:2:30:10 \
                            LEADING:3 \
                            TRAILING:3 \
                            SLIDINGWINDOW:4:15 \
                            MINLEN:36" :::: files.txt
```

2. Run FASTQC on the trimmed files, and compare with raw

```bash
parallel --gnu --eta --load 80% --j 20 --noswap fastqc -o ${OUTPUT_DIR} {} ::: ${RAW_FASTQ_DIR}/*.fastq
```

## Prepare metadata files

### Target baits file
The file ```phyloTargets.fasta``` is simply the 1,219 sequences that represent the targets sequences from the *S. purpurea* reference genome onto which we want to map our sequence capture (or WGS) reads. This includes the 972 genes for which we designed target baits, as well as 247 genes that are paralogous to genes in the bait targets but for which we were not able to design baits. We include both to ideally seperate reads from the paralogous copies in the mapping and assembly steps. **Note**: the names have a prefix "Sapur-" because Hybpiper expects gene names in this format.

### A list of gene names
The file ```genenames.txt``` contains the name of each of the 1,219 genes. **Note**: the names have a prefix "Sapur-" because Hybpiper expects gene names in this format.
### A list of sample names and FASTQ files

The file ```filelist.txt``` contains the names and prefixes for all of the libaries. For each library there are three lines, one that is the prefix for naming the directories (and I hope the labels on all downstream applications), and two that reference the file names.

The file ```namelist.txt``` contains just the names of each sample.

## Assemble gene sequences with HybPiper

I have adapted much of this pipeline from both the [HybPiper tutorial](https://github.com/mossmatters/HybPiper/wiki) as well as [the notes from a workshop](https://github.com/mossmatters/KewHybSeqWorkshop) Matt Johnson gave at Kew. 

HybPiper uses GNU parallel in order to parallelize jobs. The following batch scripts are all run on the HPCC at Texas Tech University. These scripts are provided as examples for how to run this pipeline on an HPCC, but may need to be adapted to work in different submission queue systems.

For readability I have replaced absolute paths with ```${WORK_DIR}``` and ```${SCRATCH_DIR}``` in these scripts. It is strongly recommended that HybPiper outputs to a scratch drive, because it creates an incredibly large number of temporary files.

```bash
#!/bin/sh
#$ -V
#$ -cwd
#$ -t 1-39:3
#$ -S /bin/bash
#$ -N phyloTrial
#$ -o ${WORK_DIR}/log/01/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e ${WORK_DIR}/log/01/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -q omni
#$ -pe sm 36
#$ -P quanah

# This code block uses the SGE_TASK_ID number (e.g. 1 for the first job in the task array)
# to read the lines from filelist.txt to associate the sample names with the correct
# FASTQ files as the variables $prefix, $pair1, and $pair2 below

prefixNum="$SGE_TASK_ID"
prefixNumP="$prefixNum"p
pair1Num=$(($prefixNum + 1))
pair1NumP="$pair1Num"p
pair2Num=$(($pair1Num + 1))
pair2NumP="$pair2Num"p
prefix=`sed -n "$prefixNumP" filelist.txt`
pair1=`sed -n "$pair1NumP" filelist.txt`
pair2=`sed -n "$pair2NumP" filelist.txt`

${WORK_DIR}/.bin/HybPiper/reads_first.py \
    -b ${WORK_DIR}/phyloTargets.fasta \
    -r ${SCRATCH_DIR}/reads/$pair1 /lustre/scratch/briasand/phylo/reads/$pair2 \
    --prefix ${SCRATCH_DIR}/$prefix \
    --bwa \
    --cpu 36

python ${WORK_DIR}/.bin/HybPiper/cleanup.py ${SCRATCH_DIR}/$prefix
```

### Retrieve the actual sequences 

Now that the gene and intron sequences have been assembled by HybPiper, we want to move the actual sequence files from the temporary scratch space, to a more stable location on ```${WORK_DIR}```

```bash
#!/bin/sh
#$ -V
#$ -wd ${SCRATCH_DIR}
#$ -S /bin/bash
#$ -N phyloTrial-postHyb
#$ -o ${WORK_DIR}/log/03/$JOB_NAME.o$JOB_ID
#$ -e ${WORK_DIR}/log/03/$JOB_NAME.e$JOB_ID
#$ -q omni
#$ -pe sm 1
#$ -P quanah

python ${WORK_DIR}/.bin/HybPiper/get_seq_lengths.py ${WORK_DIR}/phyloTargets.fasta ${WORK_DIR}/namelist.txt dna > ${WORK_DIR}/gene_lengths.txt

python ${WORK_DIR}/.bin/HybPiper/hybpiper_stats.py ${WORK_DIR}/gene_lengths.txt ${WORK_DIR}/namelist.txt > ${WORK_DIR}/phylo_stats.txt

python ${WORK_DIR}/.bin/HybPiper/retrieve_sequences.py ${WORK_DIR}/phyloTargets.fasta . dna
mv *.FNA ${WORK_DIR}/FNA/

python ${WORK_DIR}/.bin/HybPiper/retrieve_sequences.py ${WORK_DIR}/phyloTargets.fasta . aa
mv *.FAA ${WORK_DIR}/phyloTrial/FAA/
```

## Identify genes with potential paralogs

HybPiper identifies genes that have multiple competing assemblies that are around the same length as the primary assembly as potentially having paralogs. The following is adapted from the [HybPiper paralog tutorial](https://github.com/mossmatters/HybPiper/wiki/Paralogs).

```bash
#!/bin/sh
#$ -V
#$ -wd ${SCRATCH_DIR}
#$ -S /bin/bash
#$ -N phyloTrial-prepParalogs
#$ -o ${WORK_DIR}/log/04/$JOB_NAME.o$JOB_ID
#$ -e ${WORK_DIR}/log/04/$JOB_NAME.e$JOB_ID
#$ -q omni
#$ -pe sm 1
#$ -P quanah

while read i; do echo $i;
    python ${WORK_DIR}/.bin/HybPiper/paralog_investigator.py $i;
done < ${WORK_DIR}/namelist.txt

```

The stderr file that is generated contains a list of all of the genes with the counts of paralog warnings in them. It needs to be processed in order to be useful. It is named in part by the job number on the HPCC, so it named something like ```log/04/phylo-prepParalogs.e######```. Use the following code to generate a list of the unique genes with paralog warnings:

```bash
cat log/04/phylo-prepParalogs.e220513 | cut -f 5 -d ' ' | sort | uniq > genesWithParalogs.txt
```

## Phylogenetic analysis

### Prepare the sequence files for RAxML

This script will:
1. Convert the **\*** characters into **X** characters for stop codons
2. Prepare sequence alignments on the amino acid sequences using mafft
3. Convert the aligned amino acid sequences to codon-aligned nucleotide sequences using pal2nal
4. Trim the codon-aligned nucleotide sequences using trimal

```bash
#!/bin/sh
#$ -V
#$ -cwd
#$ -S /bin/bash
#$ -N phyloTrial-prepTreeFiles
#$ -t 1-1219:1
#$ -o ${WORK_DIR}/log/06/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e ${WORK_DIR}/log/06/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -q omni
#$ -pe sm 1
#$ -P quanah

prefixNum=$(SGE_TASK_ID)
prefixNumP=$(prefixNum)p
prefix=`sed -n $(prefixNumP) genenames.txt`

sed -i 's/*/X/g' FAA/$(prefix).FAA

mafft --localpair --maxiterate 1000 FAA/$(prefix).FAA > aligned/$(prefix).aligned.FAA

${WORK_DIR}/.bin/pal2nal.v14/pal2nal.pl -output fasta aligned/$(prefix).aligned.FAA FNA/$(prefix).FNA > inframe/$(prefix).inframe.FNA

trimal -gt 0.5 -in inframe/$(prefix).inframe.FNA -out trimmed/$(prefix).trimmed.FNA
```

### Create gene trees with RAxML

This script uses RAxML to create gene trees with 250 bootstrap replicates for all 972 genes

```bash
#!/bin/sh
#$ -V
#$ -cwd
#$ -S /bin/bash
#$ -N phyloTrial-makeTrees
#$ -t 1-1219:1
#$ -o ${WORK_DIR}/log/07/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e ${WORK_DIR}/log/07/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -q omni
#$ -pe sm 1
#$ -P quanah

prefixNum=$(SGE_TASK_ID)
prefixNumP=$(prefixNum)p
prefix=`sed -n $(prefixNumP) genenames.txt`



raxmlHPC -m GTRGAMMA -p 12345 -# 20 -s trimmed/$(prefix).trimmed.FNA -n $(prefix)
mv RAxML*$(prefix)* RAxML/bestTree/

raxmlHPC -m GTRGAMMA -p 12345 -b 12345 -# 250 -s trimmed/$(prefix).trimmed.FNA -n $(prefix)
mv RAxML*($prefix)* RAxML/bootstrap/
```